# Load manual dataset

In [ ]:
from pathlib import Path
from tifffile import imread

In [ ]:
image_paths = sorted(Path('manual_raw_v3').glob('*/images/*.tif'))
label_paths = sorted(Path('manual_raw_v3').glob('*/masks/*.tif'))

In [ ]:
assert(len(image_paths) == len(label_paths))

In [ ]:
X = [imread(image_path) for image_path in image_paths]
Y = [imread(label_paths) for label_path in label_paths]

# Create predictions

## Stardist

In [ ]:
stardist_type = 'stardist_192_48x96x96_patches-semimanual-raw-64x128x128_True_100prc_rep*'

In [ ]:
models = sorted(Path('./models').glob(stardist_type))
models

In [ ]:
from subprocess import check_output

for model in models: 
    p = check_output([
        'python', 'stardist_prediction.py',
            str(model),
            'manual_raw_v3/test/images/im0.tif',
            'stardist_predictions'])
    print(p.decode())

## Cellpose/ horovod

In [ ]:
horovod_type = 'horovod_cellpose_patches-semimanual-raw-64x128x128_prc100_bs8_lr0.00625_wd0.00001_mt0.7_sge_rep*'
models = sorted(Path('./models').glob(horovod_type))
models

In [ ]:
for model in models: 
    p = check_output([
        'python', 'cellpose_prediction.py',
            str(model),
            'manual_raw_v3/test/images/im0.tif',
            'dummy'])
    print(p.decode())

## Semi-manual annotation of manual dataset

In [ ]:
from tifffile import imsave

crop_values = (slice(10, 139), slice(255, 512), slice(255,512))
semimanual_annotation = 'napari_annotation/Pos1_ch2_frame000001_Nz300L.tif'

semimanual_annotation = imread(str(semimanual_annotation))


Y_ = [semimanual_annotation[crop_values]]


output_dir = Path('interference') / 'im0' / 'semi-manual_annotation'
output_dir.mkdir(parents=True, exist_ok=True)

filelist_x = [Path('im0P.tif')]

assert(len(filelist_x) == len(Y_))

for y, x_path in zip(Y_, filelist_x):
    imsave(output_dir / x_path.name, y, compress=9)

## Hartmann et al.

In [ ]:
from subprocess import check_output

In [ ]:
data_dir = r'Y:/Eric/2021_Iterative_Biofilm_Annotation/Figures/FigS5_PredictionVsManualGT/ExperimentData_hartmann_et_al/data'
script_folder = r'Y:/Eric/2021_Iterative_Biofilm_Annotation/Figures/FigS5_PredictionVsManualGT'
cmd = ['matlab', '-nojvm', '-nosplash', '-batch', f"addpath('{script_folder}');data2labelimage('{data_dir}','{data_dir}');"]
print(' '.join(cmd))
#output = check_output(cmd)

## Sanity check

In [ ]:
crop_values = (slice(10, 139), slice(255, 512), slice(255,512))

import numpy as np
im_raw_full_path = sorted(Path('ExperimentData_hartmann_et_al').glob('*_ch1_*.tif'))
assert(len(im_raw_full_path) == 1)
im_raw_full = imread(im_raw_full_path[0])
im_raw_cropped = im_raw_full[crop_values]
assert(np.all(im_raw_cropped == im0_test))

It looks like ch2 is +1 z offsetted in comparision to ch1:

Check the influence of different offsets on the accuracy mesurement!

In [ ]:
im_prediction_full = sorted(Path('ExperimentData_hartmann_et_al/data').glob('*.tif'))

assert(len(im_prediction_full) == 1)

im_prediction_full = imread(im_prediction_full)

print(im_prediction_full.shape, im_raw_full.shape)

from scipy.ndimage import affine_transform

new_shape = np.asarray(im_prediction_full.shape, dtype=np.float)
new_shape = new_shape * np.array([61/100, 1, 1])
new_shape, im_raw_full.shape

def ImageInterpolation(image, d, new_shape, order=3):

    trans_matrix = np.diag([1/d, 1, 1])
    rescaled = affine_transform(image, trans_matrix, output_shape=new_shape, order=order)

    return rescaled

factor = 61/100
im_rescaled = ImageInterpolation(im_prediction_full, factor, new_shape.astype(int), order=0)

In [ ]:
im_rescaled.shape, im_raw_full.shape

In [ ]:
for offset in range(-2,3):
    print(offset)
    c = crop_values[0]
    crop_values_ = (slice(c.start + offset, c.stop+offset, c.step), crop_values[1], crop_values[2])
    im0P = im_rescaled[crop_values_]
    output_file = Path(f'interference/im0/hartmann_et_al_offset{offset}/im0P.tif')
    output_file.parent.mkdir(parents=True, exist_ok=True)
    imwrite(output_file, im0P)

## Seeded Watershed (same story)

In [ ]:
data_dir = r'Y:/Eric/2021_Iterative_Biofilm_Annotation/Figures/FigS5_PredictionVsManualGT/ExperimentData_seeded_watershed/data'
script_folder = r'Y:/Eric/2021_Iterative_Biofilm_Annotation/Figures/FigS5_PredictionVsManualGT'
cmd = ['matlab', '-nojvm', '-nosplash', '-batch', f"addpath('{script_folder}');data2labelimage('{data_dir}','{data_dir}');"]
print(' '.join(cmd))
#output = check_output(cmd)

In [ ]:
crop_values = (slice(10, 139), slice(255, 512), slice(255,512))

import numpy as np
im_raw_full_path = sorted(Path('ExperimentData_seeded_watershed').glob('*_ch1_*.tif'))
assert(len(im_raw_full_path) == 1)
im_raw_full = imread(im_raw_full_path[0])
im_raw_cropped = im_raw_full[crop_values]
print(im_raw_cropped.shape, im0_test.shape)
assert(np.all(im_raw_cropped == im0_test))

In [ ]:
im_prediction_full = sorted(Path('ExperimentData_seeded_watershed/data').glob('*.tif'))

assert(len(im_prediction_full) == 1)

im_prediction_full = imread(im_prediction_full)

print(im_prediction_full.shape, im_raw_full.shape)

from scipy.ndimage import affine_transform

new_shape = np.asarray(im_prediction_full.shape, dtype=np.float)
new_shape = new_shape * np.array([61/100, 1, 1])
new_shape, im_raw_full.shape

def ImageInterpolation(image, d, new_shape, order=3):

    trans_matrix = np.diag([1/d, 1, 1])
    rescaled = affine_transform(image, trans_matrix, output_shape=new_shape, order=order)

    return rescaled

factor = 61/100
im_rescaled = ImageInterpolation(im_prediction_full, factor, new_shape.astype(int), order=0)

In [ ]:
im_rescaled.shape, im_raw_full.shape

In [ ]:
for offset in range(-2,3):
    print(offset)
    c = crop_values[0]
    crop_values_ = (slice(c.start + offset, c.stop+offset, c.step), crop_values[1], crop_values[2])
    im0P = im_rescaled[crop_values_]
    output_file = Path(f'interference/im0/seeded_watershed_offset{offset}/im0P.tif')
    output_file.parent.mkdir(parents=True, exist_ok=True)
    imwrite(output_file, im0P)

# Calculate all accuracies

In [ ]:
from tifffile import imread
from stardist.matching import  matching_dataset
import numpy as np
from pathlib import Path


def calc_accuracy(output_file, pred_path, gt_path, z_cutoff=None):
    output_file = Path(output_file)
    Y_pred_paths = [Path(pred_path)]
    Y_true_paths = [Path(gt_path)]
    
    Y_pred = [imread(str(p)) for p in Y_pred_paths]
    Y_true = [imread(str(p)) for p in Y_true_paths]

    tau_values = tuple(np.arange(0.05, 1, 0.05))
    m = matching_dataset(Y_true, Y_pred, thresh=tau_values, show_progress=True, parallel=True)

    output = np.asarray([tau_values, [stats._asdict()['accuracy'] for stats in m]])

    np.savetxt(output_file, output)


In [ ]:
predictions = sorted(Path('interference/im0').glob('*/im0P.tif'))

In [ ]:
gt_path = Path(r"Y:\Eric\2021_Iterative_Biofilm_Annotation\Figures\FigS5_PredictionVsManualGT\manual_raw_v3\test\masks\im0.tif")
for prediction in predictions:
    modelname = prediction.parent.name
    output_path = Path(f'calculated_accuracies/{modelname}/accuracy.csv')
    output_path.parent.mkdir(parents=True, exist_ok=True)
    calc_accuracy(output_path, prediction, gt_path)

# Plot the accuracies

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import os


def plot_accuracies(fig_title, figure_filename, accuracy_files):
    print(figure_filename)
    print(accuracy_files)
    
    figure_filename_path = Path(figure_filename)
    figure_filename_path.parent.mkdir(parents=True, exist_ok=True)
    
    modelnames = [Path(f).parent.name for f in accuracy_files]
    
    print('\n')
    data = []
    for filename in accuracy_files:
        print(filename)
        data.append(np.genfromtxt(filename, delimiter=' '))
    
    modelnames_clean = [m.split('_rep')[0] for m in modelnames]
    model_type_names, model_types = np.unique(modelnames_clean, return_inverse=True)
    
    f, ax = plt.subplots(1)

    for model_type in range(max(model_types)+1):
        models_of_type = np.where(model_types == model_type)[0]
        data_ = [data[i] for i in models_of_type]
        print(len(data_))
        data_ = np.asarray(data_)
        mean = np.mean(data_, axis=0)
        std = np.std(data_, axis=0)
        p, = ax.plot(mean[0], mean[1], label=model_type_names[model_type] + ' N={}'.format(len(models_of_type)))
        
        ax.fill_between(mean[0], mean[1] - std[1], mean[1] + std[1],
                 color=p.get_color(), alpha=0.2)

    ax.set_ylabel('accuracy')
    ax.set_xlabel(r'$\tau$')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    ax.grid()
    ax.set_title(fig_title)

    plt.savefig(figure_filename, bbox_inches='tight')
    plt.savefig(figure_filename.split('.')[0] + '.png', bbox_inches='tight')
    
    f, ax = plt.subplots(1)
        
    for d, l in zip(data, modelnames):
        ax.plot(d[0, :], d[1, :], label=l)
        
    ax.set_ylabel('accuracy')
    ax.set_xlabel(r'$\tau$')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    ax.grid()
    ax.set_title(fig_title)
    plt.savefig(figure_filename.split('.')[0] + '_raw.png', bbox_inches='tight')

In [ ]:
accuracy_files = []
accuracy_files.append(Path('calculated_accuracies/seeded_watershed_offset0/accuracy.csv'))
accuracy_files.append(Path('calculated_accuracies/hartmann_et_al_offset0/accuracy.csv'))
accuracy_files.append(Path(r'calculated_accuracies\semi-manual_annotation\accuracy.csv'))
accuracy_files += sorted(Path('calculated_accuracies/').glob('stardist*_rep[12345]/accuracy.csv'))
accuracy_files += sorted(Path('calculated_accuracies/').glob('horovod_cellpose*/accuracy.csv'))

In [ ]:
plot_accuracies('collected_accuracies', 'output/FigS5.svg', accuracy_files)